## Using Browser Automation with Foundry Agent

### Installing Required Libraries

In [ ]:
%pip install azure-ai-projects==2.0.0b2 openai==1.109.1 python-dotenv azure-identity

### Setting Up the Environment Variables

In [ ]:
import os
from dotenv import load_dotenv
from azure.identity import DefaultAzureCredential
from azure.ai.projects import AIProjectClient
from azure.ai.projects.models import PromptAgentDefinition, WebSearchPreviewTool, ApproximateLocation
from azure.ai.projects.models import (
    PromptAgentDefinition,
    BrowserAutomationAgentTool,
    BrowserAutomationToolParameters,
    BrowserAutomationToolConnectionParameters,
)

load_dotenv()

foundry_project_endpoint = os.getenv("FOUNDRY_PROJECT_ENDPOINT")
model_deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
browser_automation_connection_name = os.getenv("BROWSER_AUTOMATION_CONNECTION_NAME")

### Setting Up the Foundry Project Client

In [ ]:
project_client = AIProjectClient(
    endpoint=foundry_project_endpoint,
    credential=DefaultAzureCredential(),
)

### Creating the OpenAI Client

In [ ]:
openai_client = project_client.get_openai_client()

### Fetching the Connection ID for the Browser Automation Tool

In [ ]:
connection_id = ""

for connection in project_client.connections.list():
    if connection.name == browser_automation_connection_name:
        connection_id = connection.id
        break

print(f"The Browser Automation Tool Connection ID is: {connection_id}")

### Creating the Browser Automation Tool Spec

In [ ]:
tool = BrowserAutomationAgentTool(
    browser_automation_preview=BrowserAutomationToolConnectionParameters(
        project_connection_id=connection_id
    )
)

### Creating the Agent 

In [ ]:
agent = project_client.agents.create_version(
    agent_name = "browser-automation-agent",
    definition=PromptAgentDefinition(
        model=model_deployment_name,
        tools=[tool],
        instructions="You are a helpful assistant that can automate web browser tasks using the Browser Automation Tool.",
    )
)

### Creating a Conversation Object for the Agent Chat System

In [ ]:
# create a conversation to use with the agent
conversation = openai_client.conversations.create()
print(f"Created conversation with id: {conversation.id}")

### Calling our Agent

In [ ]:
user_query = """Your goal is to report the percent of Microsoft year-to-date stock price change.
            To do that, go to the website finance.yahoo.com.
            At the top of the page, you will find a search bar.
            Enter the value 'MSFT', to get information about the Microsoft stock price.
            At the top of the resulting page you will see a default chart of Microsoft stock price.
            Click on 'YTD' at the top of that chart, and report the percent value that shows up just below it."""

In [ ]:
response = openai_client.responses.create(
    conversation=conversation.id,
    extra_body = {
        "agent": {
            "name": "browser-automation-agent",
            "type": "agent_reference"
        }
    },
    input=user_query
)

print(f"Agent Response: {response.output_text}")